https://www.kaggle.com/c/state-farm-distracted-driver-detection/data

In [1]:
%matplotlib inline
from __future__ import division,print_function
import os, json
from glob import glob
import numpy as np
np.set_printoptions(precision=4, linewidth=100)
from matplotlib import pyplot as plt
import utils; reload(utils)
from utils import plots
import vgg16; reload(vgg16)
from vgg16 import Vgg16
import collections
from keras.preprocessing import image
from keras.models import Sequential
from keras.layers import Convolution2D, ZeroPadding2D, MaxPooling2D
import numpy.ma as ma
import pylab as pl
import matplotlib.cm as cm
import theano as theano

from mpl_toolkits.axes_grid1 import make_axes_locatable
def nice_imshow(ax, data, vmin=None, vmax=None, cmap=None):
    """Wrapper around pl.imshow"""
    if cmap is None:
        cmap = cm.jet
    if vmin is None:
        vmin = data.min()
    if vmax is None:
        vmax = data.max()
    divider = make_axes_locatable(ax)
    cax = divider.append_axes("right", size="5%", pad=0.05)
    im = ax.imshow(data, vmin=vmin, vmax=vmax, interpolation='nearest', cmap=cmap)
    pl.colorbar(im, cax=cax)
    
def make_mosaic(imgs, nrows, ncols, border=1):
    """
    Given a set of images with all the same shape, makes a
    mosaic with nrows and ncols
    """
    nimgs = imgs.shape[0]
    imshape = imgs.shape[1:]
    mosaic = ma.masked_all((nrows * imshape[0] + (nrows - 1) * border,
                            ncols * imshape[1] + (ncols - 1) * border),
                            dtype=np.float32)
    print(mosaic.shape)
    paddedh = imshape[0] + border
    paddedw = imshape[1] + border
    for i in xrange(nimgs):
        row = int(np.floor(i / ncols))
        col = i % ncols
        #print (i)
        #print ("row",row * paddedh, row * paddedh + imshape[0])
        #print ("col",col * paddedw, col * paddedw + imshape[1])
        #if i==36:
        #   print (imgs[i])
        mosaic[row * paddedh:row * paddedh + imshape[0],
               col * paddedw:col * paddedw + imshape[1]] = imgs[i]
    return mosaic

Using gpu device 0: Tesla K80 (CNMeM is disabled, cuDNN 5103)
/home/ubuntu/anaconda2/lib/python2.7/site-packages/theano/sandbox/cuda/__init__.py:600: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.
  warnings.warn(warn)
Using Theano backend.


### Problem statement
In this competition you are given driver images, each taken in a car with a driver doing something in the car (texting, eating, talking on the phone, makeup, reaching behind, etc). Your goal is to predict the likelihood of what the driver is doing in each picture.
https://www.kaggle.com/c/state-farm-distracted-driver-detection/data

In [22]:
samplePath = "../data/kaggle_state_farm/output/sample/"
realPath = "../data/kaggle_state_farm/output/"
testpath = "../data/kaggle_state_farm/test"
trainPath = "../data/kaggle_state_farm/train"

In [33]:
%run state_farm_reloc.py

generating valid folder
('current category', 'c2')
('current category', 'c7')
('current category', 'c3')
('current category', 'c9')
('current category', 'c1')
('current category', 'c0')
('current category', 'c5')
('current category', 'c6')
('current category', 'c4')
('current category', 'c8')
generating valid folder for sample
generating train folder for sample


In [34]:
print("Starting training")
vgg = Vgg16()
batch_size = 64
#Training on the sample
print("Started training on the sample")
batches = vgg.get_batches(samplePath+'train', batch_size=batch_size)
val_batches = vgg.get_batches(samplePath+'valid', batch_size=batch_size)
vgg.finetune(batches)
vgg.fit(batches, val_batches, nb_epoch=1)
vgg.fit(batches, val_batches, nb_epoch=1)
vgg.model.save_weights("trained_sample_2_epoch.h5")


Starting training


/home/ubuntu/anaconda2/lib/python2.7/site-packages/keras/layers/core.py:577: UserWarning: `output_shape` argument not specified for layer lambda_5 and cannot be automatically inferred with the Theano backend. Defaulting to output shape `(None, 3, 224, 224)` (same as input shape). If the expected output shape is different, specify it via the `output_shape` argument.
  .format(self.name, input_shape))


Started training on the sample
Found 2184 images belonging to 10 classes.
Found 220 images belonging to 10 classes.
Epoch 1/1
2184/2184 [==============================] - 74s - loss: 4.4171 - acc: 0.2422 - val_loss: 1.9824 - val_acc: 0.4227

In [35]:
vgg.model.optimizer.lr = 0.01
vgg.fit(batches, val_batches, nb_epoch=1)
vgg.fit(batches, val_batches, nb_epoch=1)
vgg.model.save_weights("trained_sample_4_epoch.h5")
print("Training on the sample finished")

Epoch 1/1
2184/2184 [==============================] - 74s - loss: 1.7004 - acc: 0.6401 - val_loss: 1.1390 - val_acc: 0.6636

In [36]:
print("Started training on the main test set")
batches = vgg.get_batches(trainPath, batch_size=batch_size)
val_batches = vgg.get_batches(realPath+'valid', batch_size=batch_size)
vgg.finetune(batches)
vgg.fit(batches, val_batches, nb_epoch=1)
vgg.fit(batches, val_batches, nb_epoch=1)
vgg.model.save_weights("trained_2_epoch.h5")

Started training on the main test set
Found 8748 images belonging to 10 classes.
Found 2784 images belonging to 10 classes.
Epoch 1/1
8748/8748 [==============================] - 360s - loss: 2.6588 - acc: 0.4840 - val_loss: 0.4784 - val_acc: 0.8592

In [59]:
print("Started Testing on the real test data")
testing_batch_size =4
batches = vgg.get_batches(testpath, batch_size=testing_batch_size, shuffle=False)
count = 0
mainArr = {}
result = np.array([["img","c0","c1","c2","c3","c4","c5","c6","c7","c8","c9"]])
while (count<len(batches.filenames)):
    filename = batches.filenames[count]
    filename = filename.replace("unknown/", "")
    
    fullpathName = testpath+"/"+batches.filenames[count]
    localImage = image.load_img(fullpathName, target_size=[224, 224])
    newPrediction = vgg.predict(image.img_to_array(localImage).reshape(1, 3, 224, 224))
#     print(newPrediction[0], newPrediction[1])
    classi = newPrediction[1]
    pred = newPrediction[0]
    if pred>0.9:
        pred = 0.9
    if pred<0.1:
        pred = 0.1
    arr = np.array([[filename, 0.10, 0.10, 0.10, 0.10,0.10,0.10,0.10,0.10,0.10,0.10 ]])
    arr[0,classi+1] = pred
    result =np.append(result, arr, 0)
    count = count + 1
np.savetxt("result.csv",result, delimiter=',', fmt="%s")

Started Testing on the real test data
Found 79726 images belonging to 1 classes.


In [56]:
result.shape

(22, 11)

In [57]:
np.savetxt("result.csv",result, delimiter=',', fmt="%s")

In [58]:
%cat result.csv

img,c0,c1,c2,c3,c4,c5,c6,c7,c8,c9
img_61280.jpg,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.9
img_7545.jpg,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.7282564044
img_82744.jpg,0.1,0.9,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1
img_7018.jpg,0.7849866747,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1
img_73407.jpg,0.1,0.1,0.1,0.1,0.1,0.9,0.1,0.1,0.1,0.1
img_8567.jpg,0.1,0.1,0.1,0.1,0.7087554931,0.1,0.1,0.1,0.1,0.1
img_96995.jpg,0.1,0.1,0.1,0.1,0.1,0.1,0.9,0.1,0.1,0.1
img_44190.jpg,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.88224732875
img_45564.jpg,0.1,0.1,0.1,0.1,0.85951089859,0.1,0.1,0.1,0.1,0.1
img_92839.jpg,0.1,0.1,0.1,0.1,0.9,0.1,0.1,0.1,0.1,0.1
img_68152.jpg,0.1,0.1,0.35320454835,0.1,0.1,0.1,0.1,0.1,0.1,0.1
img_60608.jpg,0.83817499876,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1
img_10305.jpg,0.1,0.1,0.1,0.1,0.1,0.56290423870,0.1,0.1,0.1,0.1
img_49079.jpg,0.1,0.1,0.1,0.1,0.1,0.9,0.1,0.1,0.1,0.1
img_78448.jpg,0.1,0.1,0.1,0.1,0.1,0.1,0.60651946067,0.1,0.1,0.1
img_12621.jpg,0.1,0.1,0.1,0.1,0.1,0.9,0.1,0.1,0.1,0.1
img_82923.jpg,0.1,